Global setup



In [4]:
n=1
k=4
q=41 #Alkaline D
#q should be divisible by 8 for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2

In [5]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [6]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [7]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.765852644390328

Expected number of repetitions:

In [9]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))^2.n()

1.70494183535004

Alice generates a signing key

In [10]:
#A = random_matrix(ZZ, 4, 4, x=0, y=q)
A = matrix(4,4, 
[[ 4, 14,  4, 14],
[27, 28,  8, 40],
[ 9,  5,  8, 19],
[ 8,  9, 14,  3]])
A

[ 4 14  4 14]
[27 28  8 40]
[ 9  5  8 19]
[ 8  9 14  3]

In [11]:
#S1 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S1 = matrix(ZZ, 
[[-1,  0, -1,  1],
[ 0,  1,  0,  1],
[ 1, -1,  0,  1],
[-1,  0,  0,  0]])
S1

[-1  0 -1  1]
[ 0  1  0  1]
[ 1 -1  0  1]
[-1  0  0  0]

In [12]:
#S2 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S2 = matrix(ZZ, 
[[ 0, -1, -1,  0],
[ 1, -1,  0, -1],
[ 0, -1,  1,  1],
[ 0,  1,  0, -1]])
S2

[ 0 -1 -1  0]
[ 1 -1  0 -1]
[ 0 -1  1  1]
[ 0  1  0 -1]

$(S_1, S_2)$ is the private key


In [13]:
T = matrix(((A*S1 + S2) % q))
T

[27  9 36 22]
[24 19 14 21]
[21 37 33 23]
[ 3 37 33 30]

In [14]:
latex(T)

\left(\begin{array}{rrrr}
27 & 9 & 36 & 22 \\
24 & 19 & 14 & 21 \\
21 & 37 & 33 & 23 \\
3 & 37 & 33 & 30
\end{array}\right)

$(A,T)$ is the public key


Alice signs the message



In [15]:
M = 'hi!'
M

'hi!'

In [16]:
#y1 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
y1 = matrix(4, 1, [-3, 0, -1, 14])
y1

[-3]
[ 0]
[-1]
[14]

In [45]:
#y2 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
y2 = matrix(4, 1, [0, -12, 9, -2])
y2

[  0]
[-12]
[  9]
[ -2]

In [46]:
w1 = matrix(((A*y1+y2) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1

[16]
[ 8]
[-6]
[ 2]

In [47]:
from sage.cpython.string import str_to_bytes

str_to_bytes(M+str(w1))

b'hi![16]\n[ 8]\n[-6]\n[ 2]'

In [48]:
import hashlib

c_hash = hashlib.shake_256(str_to_bytes(M+str(w1)))

In [91]:
c_hash.hexdigest(n*k)

'bb8289d1'

In [74]:
int(c_hash.hexdigest(3*n*k)[2], 16)

8

In [82]:
n*k

4

In [83]:
tau

1

In [85]:
def sample_in_ball(hexdigest): #from NIST proposed standard FIPS 204ipd
    # digest should be at least ~~ twice the length of c
    # using hex digits makes sense as long as the length of c is <= 16
    c = [0] * (n*k)
    ell = n*k
    for i in range(n*k-tau, n*k):
        while int(hexdigest[ell],16) > i:
            ell = ell+1
        j = int(hexdigest[ell],16)
        s = int(hexdigest[i+tau-n*k],16) % 2
        c[i] = c[j]
        c[j] = (-1)^s
    return c
    

In [92]:
c = matrix(4, 1, sample_in_ball(c_hash.hexdigest(n*k)))
c

[ 0]
[-1]
[ 0]
[ 0]

In [93]:
z1 = matrix(y1 + S1*c); z1

[-3]
[-1]
[ 0]
[14]

Is $\|\mathbf{z_1}\|_\infty \geq \gamma_1-\beta$?

In [94]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_1}$

In [95]:
z2 = matrix(y2 + S2*c); z2

[  1]
[-11]
[ 10]
[ -3]

Is $\|\mathbf{z_2}\|_\infty \geq \gamma_1-\beta$?

In [96]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_2}$

If not, $(\mathbf{z_1}, \mathbf{z_2}, \mathbf{c})$ is the signature


In [98]:
(z1, z2, c)

(
[-3]  [  1]  [ 0]
[-1]  [-11]  [-1]
[ 0]  [ 10]  [ 0]
[14], [ -3], [ 0]
)

Bob verifies the signature

Is $\|\mathbf{z_i}\|_\infty < \gamma_1-\beta$?

In [99]:
z1

[-3]
[-1]
[ 0]
[14]

In [100]:
z2

[  1]
[-11]
[ 10]
[ -3]

In [101]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [102]:
w1prime = matrix(((A*z1+z2-T*c) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1prime

[16]
[ 8]
[-6]
[ 2]

In [106]:
cprime_hash = hashlib.shake_256(str_to_bytes(M+str(w1prime)))
cprime = matrix(4, 1, sample_in_ball(cprime_hash.hexdigest(n*k)))
cprime

[ 0]
[-1]
[ 0]
[ 0]

In [107]:
c == cprime

True

Yes.  The signature is accepted.